## Google Maps Code

In [1]:
import requests
import json
import gmaps
import pandas as pd
from gkey import gkey

gmaps.configure(api_key=gkey)

In [2]:
def get_boundingbox_country(country, output_as='boundingbox'):
    """
    get the bounding box of a country in EPSG4326 given a country name

    Parameters
    ----------
    country : str
        name of the country in english and lowercase
    output_as : 'str
        chose from 'boundingbox' or 'center'. 
         - 'boundingbox' for [latmin, latmax, lonmin, lonmax]
         - 'center' for [latcenter, loncenter]

    Returns
    -------
    output : list
        list with coordinates as str
    """
    # create url
    url = '{0}{1}{2}'.format('http://nominatim.openstreetmap.org/search?country=',
                             country,
                             '&format=json&polygon=0')
    
    response_object = requests.get(url)
    
    # error handling
    if len(response_object.json())==0:
        
        return [None, None]
    else: 
        response = response_object.json()[0]

    # parse response to list
    if output_as == 'boundingbox':
        lst = response[output_as]
        output = [float(i) for i in lst]
    if output_as == 'center':
        lst = [response.get(key) for key in ['lat','lon']]
        output = [float(i) for i in lst]
    return output

In [3]:
country_df = pd.read_csv('./Datasets/Box_Office_by_Country.csv')
country_box_office_df = pd.DataFrame(country_df)
country_box_office_df

,Country,Wonder Woman,Amazing Spider Man,Amazing Spider Man 2,Ant Man,Ant Man and The Wasp,Aquaman,Black Panther,Captain America,Captain America 2,...,Logan,Man of Steel,Punisher,Spider Man: Homecoming,The Wolverine,Thor,Thor 2,Thor 3,Venom,Wolverine: Origins
0,Argentina,3390252,3357295.0,NaN,4126139,2250863,3.441474e+06,5720345,NaN,NaN,...,5148269.0,NaN,NaN,7033029,NaN,NaN,5475083.0,6169643,2830282,NaN
1,Australia,24683719,8654635.0,NaN,11681785,14348561,2.950000e+07,31932114,NaN,NaN,...,17295477.0,21880008.0,NaN,20264403,NaN,NaN,21446060.0,26997468,15667184,NaN
2,Brazil,33500000,13121674.0,24869494.0,12431089,13010094,3.620000e+07,36942424,NaN,28317546.0,...,29218096.0,NaN,NaN,31896084,NaN,12092872.0,27686492.0,30485848,19031136,NaN
3,Bulgaria,310618,NaN,385465.0,183235,224148,9.897640e+05,636460,NaN,NaN,...,295183.0,NaN,NaN,328580,379034.0,NaN,879675.0,754537,859396,NaN
4,China,155498909,NaN,NaN,105718824,120770000,2.918000e+08,113698530,NaN,NaN,...,105962992.0,NaN,NaN,1125361644,NaN,NaN,NaN,119229970,270034912,NaN
5,Czech Republic,605248,269795.0,NaN,576130,1211659,2.675829e+06,2100955,NaN,NaN,...,1102637.0,NaN,NaN,1284993,NaN,NaN,1378030.0,3044746,2700089,NaN
6,France,16300000,8589382.0,22738176.0,12674429,14715801,2.960000e+07,33114492,NaN,NaN,...,15456133.0,NaN,NaN,18790172,NaN,NaN,22289092.0,21262792,18420064,NaN
7,Germany,8700000,2732820.0,15932503.0,5917308,9424892,2.470000e+07,23392484,NaN,NaN,...,9880658.0,NaN,NaN,11671525,NaN,NaN,20356156.0,19036144,16029204,NaN
8,India,2568493,5747192.0,13603981.0,1649781,6400000,7.800000e+06,10458639,NaN,NaN,...,6565038.0,NaN,NaN,11403714,NaN,NaN,3598734.0,11818410,5927228,NaN
9,Indonesia,8600000,6708221.0,NaN,8386902,10000000,1.750000e+07,12500000,NaN,NaN,...,2900000.0,NaN,NaN,12335003,NaN,NaN,7402867.0,11746294,10704633,NaN


In [4]:
country_name = country_df['Country']

country_lat_lng_array = []

for country in country_name:
    lat, lng = get_boundingbox_country(country=country, output_as='center')
    country_lat_lng_array.append([country, lat, lng])

country_lat_lng_df = pd.DataFrame(country_lat_lng_array)

In [5]:
country_lat_lng_df.columns = ['Country', 'Lat', 'Lng']
country_box_office_lat_lng = pd.merge(country_lat_lng_df, country_box_office_df, on ="Country")

country_box_office_lat_lng.loc[country_box_office_lat_lng['Country'] == 'Philippines', ['Lat']] = 12.8797
country_box_office_lat_lng.loc[country_box_office_lat_lng['Country'] == 'Philippines', ['Lng']] = 121.7740

country_box_office_lat_lng.drop([26,28], inplace=True)
country_box_office_lat_lng

,Country,Lat,Lng,Wonder Woman,Amazing Spider Man,Amazing Spider Man 2,Ant Man,Ant Man and The Wasp,Aquaman,Black Panther,...,Logan,Man of Steel,Punisher,Spider Man: Homecoming,The Wolverine,Thor,Thor 2,Thor 3,Venom,Wolverine: Origins
0,Argentina,-34.996496,-64.967282,3390252,3357295.0,NaN,4126139,2250863,3441474.0,5720345,...,5148269.0,NaN,NaN,7033029,NaN,NaN,5475083.0,6169643,2830282,NaN
1,Australia,-24.776109,134.755000,24683719,8654635.0,NaN,11681785,14348561,29500000.0,31932114,...,17295477.0,21880008.0,NaN,20264403,NaN,NaN,21446060.0,26997468,15667184,NaN
2,Brazil,-10.333333,-53.200000,33500000,13121674.0,24869494.0,12431089,13010094,36200000.0,36942424,...,29218096.0,NaN,NaN,31896084,NaN,12092872.0,27686492.0,30485848,19031136,NaN
3,Bulgaria,42.607397,25.485662,310618,NaN,385465.0,183235,224148,989764.0,636460,...,295183.0,NaN,NaN,328580,379034.0,NaN,879675.0,754537,859396,NaN
4,China,35.000074,104.999927,155498909,NaN,NaN,105718824,120770000,291800000.0,113698530,...,105962992.0,NaN,NaN,1125361644,NaN,NaN,NaN,119229970,270034912,NaN
5,Czech Republic,49.816700,15.474954,605248,269795.0,NaN,576130,1211659,2675829.0,2100955,...,1102637.0,NaN,NaN,1284993,NaN,NaN,1378030.0,3044746,2700089,NaN
6,France,46.603354,1.888334,16300000,8589382.0,22738176.0,12674429,14715801,29600000.0,33114492,...,15456133.0,NaN,NaN,18790172,NaN,NaN,22289092.0,21262792,18420064,NaN
7,Germany,51.083420,10.423447,8700000,2732820.0,15932503.0,5917308,9424892,24700000.0,23392484,...,9880658.0,NaN,NaN,11671525,NaN,NaN,20356156.0,19036144,16029204,NaN
8,India,22.351115,78.667743,2568493,5747192.0,13603981.0,1649781,6400000,7800000.0,10458639,...,6565038.0,NaN,NaN,11403714,NaN,NaN,3598734.0,11818410,5927228,NaN
9,Indonesia,-2.495793,120.200072,8600000,6708221.0,NaN,8386902,10000000,17500000.0,12500000,...,2900000.0,NaN,NaN,12335003,NaN,NaN,7402867.0,11746294,10704633,NaN


In [6]:
ant_man_and_wasp_df = pd.DataFrame(country_box_office_lat_lng[['Country', 'Lat', 'Lng', 'Ant Man and The Wasp']])

black_panther_df = pd.DataFrame(country_box_office_lat_lng[['Country', 'Lat', 'Lng', 'Black Panther']])

dark_knight_rises_df = pd.DataFrame(country_box_office_lat_lng[['Country', 'Lat', 'Lng', 'Dark Knight Rises']])

iron_man_3_df = pd.DataFrame(country_box_office_lat_lng[['Country', 'Lat', 'Lng', 'Iron Man 3']])

spiderman_homecoming_df = pd.DataFrame(country_box_office_lat_lng[['Country', 'Lat', 'Lng', 'Spider Man: Homecoming']])

thor_3_df = pd.DataFrame(country_box_office_lat_lng[['Country', 'Lat', 'Lng', 'Thor 3']])

venom_df = pd.DataFrame(country_box_office_lat_lng[['Country', 'Lat', 'Lng', 'Venom']])

wonder_woman_df = pd.DataFrame(country_box_office_lat_lng[['Country', 'Lat', 'Lng', 'Wonder Woman']])

In [7]:
ant_man_and_wasp_locations = ant_man_and_wasp_df[['Lat', 'Lng']]
ant_man_and_wasp_sales = ant_man_and_wasp_df['Ant Man and The Wasp'].astype(float)

ant_man_and_wasp_fig = gmaps.figure()
ant_man_and_wasp_heat_layer = gmaps.heatmap_layer(ant_man_and_wasp_locations, weights=ant_man_and_wasp_sales, 
                                 dissipating=False, max_intensity=ant_man_and_wasp_sales.max()/2)

ant_man_and_wasp_fig.add_layer(ant_man_and_wasp_heat_layer)

ant_man_and_wasp_fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
spiderman_homecoming_locations = spiderman_homecoming_df[['Lat', 'Lng']]
spiderman_homecoming_sales = spiderman_homecoming_df['Spider Man: Homecoming'].astype(float)

spiderman_homecoming_fig = gmaps.figure()
spiderman_homecoming_heat_layer = gmaps.heatmap_layer(spiderman_homecoming_locations, weights=spiderman_homecoming_sales, 
                                 dissipating=False, max_intensity=spiderman_homecoming_sales.max()/2)

spiderman_homecoming_fig.add_layer(spiderman_homecoming_heat_layer)

spiderman_homecoming_fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
black_panther_locations = black_panther_df[['Lat', 'Lng']]
black_panther_sales = black_panther_df['Black Panther'].astype(float)

black_panther_fig = gmaps.figure()
black_panther_heat_layer = gmaps.heatmap_layer(black_panther_locations, weights=black_panther_sales, 
                                 dissipating=False, max_intensity=black_panther_sales.max()/2)

black_panther_fig.add_layer(black_panther_heat_layer)

black_panther_fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
dark_knight_rises_locations = dark_knight_rises_df[['Lat', 'Lng']]
dark_knight_rises_sales = dark_knight_rises_df['Dark Knight Rises'].astype(float)

dark_knight_rises_fig = gmaps.figure()
dark_knight_rises_heat_layer = gmaps.heatmap_layer(dark_knight_rises_locations, weights=dark_knight_rises_sales, 
                                 dissipating=False, max_intensity=dark_knight_rises_sales.max()/2)

dark_knight_rises_fig.add_layer(dark_knight_rises_heat_layer)

dark_knight_rises_fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
iron_man_3_locations = iron_man_3_df[['Lat', 'Lng']]
iron_man_3_sales = iron_man_3_df['Iron Man 3'].astype(float)

iron_man_3_fig = gmaps.figure()
iron_man_3_heat_layer = gmaps.heatmap_layer(iron_man_3_locations, weights=iron_man_3_sales, 
                                 dissipating=False, max_intensity=iron_man_3_sales.max()/2)

iron_man_3_fig.add_layer(iron_man_3_heat_layer)

iron_man_3_fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
thor_3_locations = thor_3_df[['Lat', 'Lng']]
thor_3_sales = thor_3_df['Thor 3'].astype(float)

thor_3_fig = gmaps.figure()
thor_3_heat_layer = gmaps.heatmap_layer(thor_3_locations, weights=thor_3_sales, 
                                 dissipating=False, max_intensity=iron_man_3_sales.max()/2)

thor_3_fig.add_layer(thor_3_heat_layer)

thor_3_fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
venom_locations = venom_df[['Lat', 'Lng']]
venom_sales = venom_df['Venom'].astype(float)

venom_fig = gmaps.figure()
venom_heat_layer = gmaps.heatmap_layer(venom_locations, weights=venom_sales, 
                                 dissipating=False, max_intensity=venom_sales.max()/2)

venom_fig.add_layer(venom_heat_layer)

venom_fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
wonder_woman_locations = wonder_woman_df[['Lat', 'Lng']]
wonder_woman_sales = wonder_woman_df['Wonder Woman'].astype(float)

wonder_woman_fig = gmaps.figure()
wonder_woman_heat_layer = gmaps.heatmap_layer(wonder_woman_locations, weights=wonder_woman_sales, 
                                 dissipating=False, max_intensity=wonder_woman_sales.max()/2)

wonder_woman_fig.add_layer(wonder_woman_heat_layer)

wonder_woman_fig

Figure(layout=FigureLayout(height='420px'))